In [18]:
import sys

In [19]:
%env CITYSCAPES_DATASET = /home/rvygon/data/

env: CITYSCAPES_DATASET=/home/rvygon/data/


In [27]:
import ImportUtil
initTrain()
X_train, y_train = importBatch(500,0)

ImportError: cannot import name '_validate_lengths'